/opt/miniconda3/envs/BigData_GerenciaTI/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 193k/193k [00:00<00:00, 949kB/s]

Extracting files...
Path to dataset files: /Users/sebastiangomba/.cache/kagglehub/datasets/programmer3/cloud-resource-usage-dataset-for-anomaly-detection/versions/1


In [4]:
import pandas as pd
df=pd.read_csv("cloud_dataset.csv")
df.head()

,Timestamp,CPU_Usage,Memory_Usage,Disk_IO,Network_IO,Workload_Type,User_ID,Anomaly_Label
0,2025-07-01 00:00:00,18.88,43.19,11.40,6.01,Database_Query,user_1,0
1,2025-07-01 00:01:00,25.31,45.43,7.68,17.67,Video_Streaming,user_1,0
2,2025-07-01 00:02:00,3.87,49.50,14.08,3.48,Database_Query,user_1,0
3,2025-07-01 00:03:00,20.92,25.88,17.33,4.77,Web_Service,user_1,0
4,2025-07-01 00:04:00,55.59,43.94,10.61,4.48,Web_Service,user_1,0


In [ ]:
print(df.isnull().sum())

df = df.dropna()  


# 4. Detección de valores duplicados
print("Duplicados:", df.duplicated().sum())
df = df.drop_duplicates()

# 5. Detección y eliminación de valores extremos (outliers)
# Por ejemplo, para CPU, memoria, red y disco
metrics = ['CPU_Usage', 'Memory_Usage', 'Disk_IO', 'Network_IO']

for col in metrics:
    mean = df[col].mean()
    std = df[col].std()
    upper_limit = mean + 3 * std
    lower_limit = mean - 3 * std
    df = df[(df[col] >= lower_limit) & (df[col] <= upper_limit)]


df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df = df.dropna(subset=['Timestamp'])

print(df['Workload_Type'].value_counts())
print(df['Anomaly_Label'].value_counts())


df.to_csv("clean_cloud_usage.csv", index=False)

Timestamp        0
CPU_Usage        0
Memory_Usage     0
Disk_IO          0
Network_IO       0
Workload_Type    0
User_ID          0
Anomaly_Label    0
dtype: int64
Duplicados: 0
Workload_Type
Web_Service        4306
Database_Query     4292
Video_Streaming    2521
Crypto_Mining      1351
Backup              135
Name: count, dtype: int64
Anomaly_Label
0    11463
1     1142
Name: count, dtype: int64


In [9]:
df["Anomaly_Label"].value_counts()

Anomaly_Label
0    11463
1     1142
Name: count, dtype: int64